In [7]:
%load_ext autoreload
%autoreload 2
import os
import re
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from scipy.io.wavfile import read, write
from pipefunc.pipefunc import do_command, send_command, get_response, get_response_json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
p = Path(os.getcwd())
p_raw = p.parents[0].joinpath('raw')
p_mono = p.parents[0].joinpath('mono')
p_mono.mkdir(exist_ok=True)
wavfilelist = sorted(p_raw.glob('*.wav'))
print(wavfilelist)
op_monofilelist = [p_mono.joinpath(x.name) for x in wavfilelist]
# print(op_monofilelist)

[WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP019/raw/Aileen_Aileen.wav'), WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP019/raw/Aileen_Mike.wav'), WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP019/raw/Aileen_Robin.wav'), WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP019/raw/Aileen_WWL.wav')]


In [4]:
if p_raw.joinpath('metadata.json').is_file():
    with open(p_raw.joinpath('metadata.json'), 'r') as f:
        metadata_dict = json.load(f)
else:    
    metadata_dict = {}
    for wavfile in wavfilelist:
        print(wavfile)
        fs, data = read(wavfile)
        print(fs)
        print(data.shape)
        metadata_dict[wavfile.name] = {'fs': fs, 
                                        'shape': data.shape,
                                        }
    with open(p_raw.joinpath('metadata.json'), 'w') as f:
        json.dump(metadata_dict, f)

print(metadata_dict)
audiolength = []
for wavfile, value in metadata_dict.items():
    audiolength.append(value['shape'][0])
print(audiolength)
maxlength = np.amax(audiolength)

for wavfile, op_monofile in zip(wavfilelist, op_monofilelist):
    if not op_monofile.is_file():
        fs, data = read(wavfile)
        data_tmp = np.pad(data[:, 0], (0, maxlength - data.shape[0]), 'constant')
        write(op_monofile, fs, data[:, 0])


{'Aileen_Aileen.wav': {'fs': 44100, 'shape': [213447849, 2]}, 'Aileen_Mike.wav': {'fs': 44100, 'shape': [213447849, 2]}, 'Aileen_Robin.wav': {'fs': 44100, 'shape': [213447849, 2]}, 'Aileen_WWL.wav': {'fs': 44100, 'shape': [213450620, 2]}}
[213447849, 213447849, 213447849, 213450620]


In [32]:
def getinfo2json():
    send_command('GetInfo: Format="JSON" Type="Tracks"')
    response = get_response_json()
    result = json.loads(response)
    return(result)

trackinfo = getinfo2json()
print(trackinfo)

filename_noext = [re.sub('.wav', '', wavfile.name) for wavfile in wavfilelist]

namelistorder = ['WWL', 'Aileen', 'Robin', 'Mike']
tracknamelist = [trackinfo['name'] for trackinfo in trackinfo]
tracklengthlist = [trackinfo['end'] - trackinfo['start'] for trackinfo in trackinfo]
name_tag = 'Aileen'

def importfiles(filepath):
    do_command(f'Import2: Filename={filepath}')

TRACK_HEIGHT = 80
TRACK_OFFSET = 29.6
MATERIAL_DIR = Path('Z:\sc2.shih\Drive\Podcast\Edit\material')
OPMUSIC_PATH = MATERIAL_DIR.joinpath('direct_import', 'direct_import_op.wav')
ENDMUSIC_PATH = MATERIAL_DIR.joinpath('direct_import', 'direct_import_end.wav')
ENDCREDIT_PATH = MATERIAL_DIR.joinpath('direct_import', 'direct_import_endcredit.wav')
ENDCREDIT_OFFSET = 9.2

for idx, name in enumerate(namelistorder):
    track_name = f'{name_tag}_{name}'
    if track_name not in tracknamelist:
        print(f'import track: {track_name}')
        importfiles(p_mono.joinpath(f'{track_name}.wav'))
    do_command('SelectAll:')        
    do_command(f'SelectTracks: Mode="Set" Track="{idx}" TrackCouint="1"')
    do_command('ZoomSel')
    do_command(f'SetTrack: Height={str(TRACK_HEIGHT)}')
    do_command(f'SetClip: At="0" Start="{TRACK_OFFSET}"')

trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]
TRACK_LENGTH_MAX = np.amax(tracklengthlist)

if 'opmusic' not in tracknamelist:
    do_command(f'Import2: Filename="{OPMUSIC_PATH}"')
    do_command(f'SelectTracks: Mode="Set" Track="{len(tracknamelist)}" TrackCouint="1"')
    do_command(f'SetTrack: Name="opmusic" Height={int(TRACK_HEIGHT*0.6)}')

trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]

if 'endmusic' not in tracknamelist:
    do_command(f'Import2: Filename="{ENDMUSIC_PATH}"')
    do_command(f'SelectTracks: Mode="Set" Track="{len(tracknamelist)}" TrackCouint="1"')
    do_command(f'SetTrack: Name="endmusic" Height={int(TRACK_HEIGHT*0.6)}')
    do_command(f'SetClip: At="0" Start="{TRACK_OFFSET + TRACK_LENGTH_MAX}"')

trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]

if 'endcredit' not in tracknamelist:
    do_command(f'Import2: Filename="{ENDCREDIT_PATH}"')
    do_command(f'SelectTracks: Mode="Set" Track="{len(tracknamelist)}" TrackCouint="1"')
    do_command(f'SetTrack: Name="endcredit" Height={int(TRACK_HEIGHT*0.6)}')
    do_command(f'SetClip: At="0" Start="{TRACK_OFFSET + TRACK_LENGTH_MAX + ENDCREDIT_OFFSET}"')



Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
[]


ValueError: zero-size array to reduction operation maximum which has no identity